In [7]:
import mysql.connector
con = mysql.connector.connect(
    host="localhost",
    user="root",
    password="password"
)
cursor = con.cursor()
query = "use DATASPARK_PROJECT"
cursor.execute(query)

In [9]:
# 1.	Demographic Distribution: Analyzing the distribution of customers based on gender, age (calculated from birthday), 
# location (city, state, country, continent).
# Here analysing the distribution of Male customers from Dallas, United States

query = """SELECT 
    CustomerKey, 
    Name,
    Gender,
    FLOOR(DATEDIFF(CURDATE(), Birthday) / 365.25) AS age, 
    City, 
    State_Customer, 
    Country_Customer, 
    Continent 
FROM 
    customers 
WHERE 
    Gender = 'Male' AND
    FLOOR(DATEDIFF(CURDATE(), Birthday) / 365.25) >= 20 AND
    FLOOR(DATEDIFF(CURDATE(), Birthday) / 365.25) <= 30 AND
    City = 'Dallas';"""
cursor.execute(query)

results = cursor.fetchall()

# Fetching the column names
column_names = [i[0] for i in cursor.description]

# Printing the column names
print(column_names)
for row in results:
    print(row)

['CustomerKey', 'Name', 'Gender', 'age', 'City', 'State_Customer', 'Country_Customer', 'Continent']
(1233069, 'Lance Barton', 'Male', 23, 'Dallas', 'Texas', 'United States', 'North America')
(1462224, 'Tracy Richter', 'Male', 25, 'Dallas', 'Texas', 'United States', 'North America')
(1498245, 'Darrell Roe', 'Male', 22, 'Dallas', 'Texas', 'United States', 'North America')
(1501383, 'William Romriell', 'Male', 28, 'Dallas', 'Texas', 'United States', 'North America')
(1848776, 'Eugene Kyle', 'Male', 27, 'Dallas', 'Texas', 'United States', 'North America')
(2004240, 'Yamal Navarrete', 'Male', 23, 'Dallas', 'Texas', 'United States', 'North America')


In [10]:
# 2. Purchase Patterns: Identifing purchasing patterns such as average order value, frequency of purchases, 
# and preferred products.

query = """-- Calculate the total number of customers
WITH TotalCustomers AS (
    SELECT COUNT(*) AS TotalCustomerCount
    FROM customers
),

-- Calculate total order value and total number of products purchased by all customers
CustomerOrderStats AS (
    SELECT
        s.CustomerKey,
        SUM(p.Unit_Price_USD * s.Quantity) AS TotalOrderValue,
        SUM(s.Quantity) AS TotalProductsPurchased
    FROM
        sales s
        JOIN products p ON s.ProductKey = p.ProductKey
    GROUP BY s.CustomerKey
),

-- Calculate the name of the product with the highest number of purchases
MostPurchasedProduct AS (
    SELECT
        p.Product_Name,
        SUM(s.Quantity) AS TotalPurchases
    FROM
        sales s
        JOIN products p ON s.ProductKey = p.ProductKey
    GROUP BY p.Product_Name
    ORDER BY TotalPurchases DESC
    LIMIT 1
)

-- Final select to get the desired results
SELECT
    (SELECT SUM(cos.TotalOrderValue) FROM CustomerOrderStats cos) / (SELECT TotalCustomerCount FROM TotalCustomers) AS AverageOrderValuePerCustomer,
    (SELECT SUM(cos.TotalProductsPurchased) FROM CustomerOrderStats cos) / (SELECT TotalCustomerCount FROM TotalCustomers) AS AverageItemsPurchasedPerCustomer,
    (SELECT Product_Name FROM MostPurchasedProduct) AS MostPurchasedProduct;"""

cursor.execute(query)

results = cursor.fetchall()

# Fetching the column names
column_names = [i[0] for i in cursor.description]

# Printing the column names
print(column_names)
for row in results:
    print(row)

['AverageOrderValuePerCustomer', 'AverageItemsPurchasedPerCustomer', 'MostPurchasedProduct']
(Decimal('3652.265138'), Decimal('12.9541'), 'WWI Desktop PC2.33 X2330 Black')


In [11]:
# 3. Segmentation: Segment customers based on demographics and purchasing behavior to identify key customer groups.
# here we are finding customerkey, Name of customers who are from 'United States', is between 20-30 years old, 
# spends more than average order value per customer and who purchases more than average number of items purchased per customer.

query = '''WITH 
-- Calculate age
customer_age AS (
    SELECT 
        CustomerKey, 
        Name,
        FLOOR(DATEDIFF(CURDATE(), Birthday) / 365.25) AS Age
    FROM customers
),

-- Calculate average order value per customer
avg_order_value AS (
    SELECT 
        AVG(total_spent) AS AverageOrderValuePerCustomer
    FROM (
        SELECT 
            s.CustomerKey, 
            SUM(s.Quantity * p.Unit_Price_USD) AS total_spent
        FROM sales s
        JOIN products p ON s.ProductKey = p.ProductKey
        GROUP BY s.CustomerKey
    ) customer_totals
),

-- Calculate average items purchased per customer
avg_items_purchased AS (
    SELECT 
        AVG(total_items) AS AverageItemsPurchasedPerCustomer
    FROM (
        SELECT 
            CustomerKey, 
            SUM(Quantity) AS total_items
        FROM sales
        GROUP BY CustomerKey
    ) customer_items
),

-- Calculate total spent and items purchased per customer
customer_totals AS (
    SELECT 
        s.CustomerKey,
        SUM(s.Quantity) AS TotalQuantityPurchased,
        SUM(s.Quantity * p.Unit_Price_USD) AS TotalAmountSpent
    FROM sales s
    JOIN products p ON s.ProductKey = p.ProductKey
    GROUP BY s.CustomerKey
)

-- Main query
SELECT DISTINCT 
    c.Name, 
    c.CustomerKey, 
    ca.Age, 
    c.Country_Customer,
    ct.TotalQuantityPurchased AS QuantityPurchased,
    ct.TotalAmountSpent AS AmountSpent
FROM customers c
JOIN customer_age ca ON c.CustomerKey = ca.CustomerKey
JOIN customer_totals ct ON c.CustomerKey = ct.CustomerKey
CROSS JOIN avg_order_value aov
CROSS JOIN avg_items_purchased aip
WHERE 
    c.Country_Customer = 'United States'
    AND ca.Age BETWEEN 30 AND 40
    AND ct.TotalAmountSpent > aov.AverageOrderValuePerCustomer
    AND ct.TotalQuantityPurchased > aip.AverageItemsPurchasedPerCustomer
ORDER BY c.CustomerKey;
'''

cursor.execute(query)

results = cursor.fetchall()

# Fetching the column names
column_names = [i[0] for i in cursor.description]

# Printing the column names
print(column_names)
for row in results:
    print(row)

['Name', 'CustomerKey', 'Age', 'Country_Customer', 'QuantityPurchased', 'AmountSpent']
('Derek Moon', 1202604, 30, 'United States', Decimal('25'), Decimal('13254.98'))
('David Thiele', 1203222, 36, 'United States', Decimal('30'), Decimal('23802.09'))
('David Gonzalez', 1208003, 37, 'United States', Decimal('35'), Decimal('14143.72'))
('Matthew Lint', 1208471, 38, 'United States', Decimal('28'), Decimal('5730.82'))
('Jean Vega', 1208615, 40, 'United States', Decimal('56'), Decimal('17980.03'))
('Betty Coogan', 1226164, 34, 'United States', Decimal('20'), Decimal('6967.66'))
('Peter Minor', 1226752, 37, 'United States', Decimal('18'), Decimal('10971.90'))
('Dianne Avalos', 1228187, 35, 'United States', Decimal('22'), Decimal('7348.95'))
('Luther Horton', 1235559, 37, 'United States', Decimal('27'), Decimal('5069.24'))
('Roxie Minton', 1243894, 36, 'United States', Decimal('38'), Decimal('24683.30'))
('Tony Dee', 1246530, 35, 'United States', Decimal('20'), Decimal('7982.75'))
('Colette L

In [12]:
# 4. Sales by Product: Evaluate which products are the top performers in terms of quantity sold and revenue generated.
# below is the SQL query to get the product name, product key, and total quantity sold for the top 5 products sold by quantity
query = """SELECT 
    p.Product_Name,
    p.ProductKey,
    SUM(s.Quantity) AS TotalQuantitySold
FROM 
    products p
JOIN 
    sales s ON p.ProductKey = s.ProductKey
GROUP BY 
    p.ProductKey, p.Product_Name
ORDER BY 
    TotalQuantitySold DESC
LIMIT 5;"""

cursor.execute(query)

results = cursor.fetchall()

# Fetching the column names
column_names = [i[0] for i in cursor.description]

# Printing the column names
print(column_names)
for row in results:
    print(row)

['Product_Name', 'ProductKey', 'TotalQuantitySold']
('WWI Desktop PC2.33 X2330 Black', 444, Decimal('550'))
('WWI Desktop PC1.80 E1800 White', 458, Decimal('538'))
('Adventure Works Desktop PC1.60 ED160 Black', 424, Decimal('521'))
('Adventure Works Desktop PC2.30 MD230 White', 434, Decimal('521'))
('Adventure Works Desktop PC1.80 ED180 Black', 425, Decimal('520'))


In [15]:
# 5. below is the SQL query to get the product name, product key, and total revenue generated for the top 5 products by revenue.
query = """SELECT 
    p.Product_Name,
    p.ProductKey,
    SUM(s.Quantity * p.Unit_Price_USD) AS TotalRevenue_in_USD
FROM 
    products p
JOIN 
    sales s ON p.ProductKey = s.ProductKey
GROUP BY 
    p.ProductKey, p.Product_Name
ORDER BY 
    TotalRevenue_in_USD DESC
LIMIT 5;"""

cursor.execute(query)

results = cursor.fetchall()

# Fetching the column names
column_names = [i[0] for i in cursor.description]

# Printing the column names
print(column_names)
for row in results:
    print(row)

['Product_Name', 'ProductKey', 'TotalRevenue_in_USD']
('WWI Desktop PC2.33 X2330 Black', 444, Decimal('505450.00'))
('Adventure Works Desktop PC2.33 XD233 Silver', 416, Decimal('466089.00'))
('Adventure Works Desktop PC2.33 XD233 Brown', 428, Decimal('464151.00'))
('Adventure Works Desktop PC2.33 XD233 Black', 422, Decimal('447678.00'))
('Adventure Works Desktop PC2.33 XD233 White', 433, Decimal('437019.00'))


In [16]:
# 6. Sales by Store: Assess the performance of different stores based on sales data.
# MySQL query to get the StoreKey, Country_Store, and total revenue for the top 5 stores by total revenue

query = """SELECT 
    s.StoreKey,
    st.Country_Store,
    SUM(s.Quantity * p.Unit_Price_USD) AS TotalRevenue
FROM 
    sales s
JOIN 
    products p ON s.ProductKey = p.ProductKey
JOIN 
    stores st ON s.StoreKey = st.StoreKey
GROUP BY 
    s.StoreKey, st.Country_Store
ORDER BY 
    TotalRevenue DESC
LIMIT 5;
"""
cursor.execute(query)

results = cursor.fetchall()

# Fetching the column names
column_names = [i[0] for i in cursor.description]

# Printing the column names
print(column_names)
for row in results:
    print(row)

['StoreKey', 'Country_Store', 'TotalRevenue']
(0, 'Online', Decimal('11404324.63'))
(55, 'United States', Decimal('2835770.82'))
(50, 'United States', Decimal('2789476.12'))
(54, 'United States', Decimal('2768792.48'))
(9, 'Canada', Decimal('2672300.12'))


In [22]:
# 7. Product Popularity: Identify the most and least popular products based on sales data.

# identifying most popular product based on total revenue generated
query = """SELECT 
    p.Product_Name,
    p.ProductKey,
    SUM(s.Quantity * p.Unit_Price_USD) AS TotalRevenue
FROM 
    products p
JOIN 
    sales s ON p.ProductKey = s.ProductKey
GROUP BY 
    p.ProductKey, p.Product_Name
ORDER BY 
    TotalRevenue DESC
LIMIT 5;"""

cursor.execute(query)

results = cursor.fetchall()

# Fetching the column names
column_names = [i[0] for i in cursor.description]

# Printing the column names
print(column_names)
for row in results:
    print(row)

['Product_Name', 'ProductKey', 'TotalRevenue']
('WWI Desktop PC2.33 X2330 Black', 444, Decimal('505450.00'))
('Adventure Works Desktop PC2.33 XD233 Silver', 416, Decimal('466089.00'))
('Adventure Works Desktop PC2.33 XD233 Brown', 428, Decimal('464151.00'))
('Adventure Works Desktop PC2.33 XD233 Black', 422, Decimal('447678.00'))
('Adventure Works Desktop PC2.33 XD233 White', 433, Decimal('437019.00'))


In [23]:
# 8. Product Popularity: Identify the most and least popular products based on sales data.

# identifying least popular product based on total revenue generated
query = """SELECT 
    p.Product_Name,
    p.ProductKey,
    SUM(s.Quantity * p.Unit_Price_USD) AS TotalRevenue
FROM 
    products p
JOIN 
    sales s ON p.ProductKey = s.ProductKey
GROUP BY 
    p.ProductKey, p.Product_Name
ORDER BY 
    TotalRevenue ASC
LIMIT 5;"""

cursor.execute(query)

results = cursor.fetchall()

# Fetching the column names
column_names = [i[0] for i in cursor.description]

# Printing the column names
print(column_names)
for row in results:
    print(row)

['Product_Name', 'ProductKey', 'TotalRevenue']
('SV USB Data Cable E600 Silver', 921, Decimal('15.20'))
('SV USB Sync Charge Cable E700 Silver', 926, Decimal('15.92'))
('Litware 80mm Dual Ball Bearing Case Fan E1001 Green', 2445, Decimal('19.96'))
('SV USB Data Cable E600 Grey', 922, Decimal('21.85'))
('SV USB Data Cable E600 Pink', 919, Decimal('25.65'))


In [25]:
# 9. Product Popularity: Identify the most and least popular products based on sales data.

# identifying most popular product based on total quantity sold

query = """SELECT 
    p.Product_Name,
    p.ProductKey,
    SUM(s.Quantity) AS TotalQuantitySold
FROM 
    products p
JOIN 
    sales s ON p.ProductKey = s.ProductKey
GROUP BY 
    p.ProductKey, p.Product_Name
ORDER BY 
    TotalQuantitySold DESC
LIMIT 5;"""

cursor.execute(query)

results = cursor.fetchall()

# Fetching the column names
column_names = [i[0] for i in cursor.description]

# Printing the column names
print(column_names)
for row in results:
    print(row)

['Product_Name', 'ProductKey', 'TotalQuantitySold']
('WWI Desktop PC2.33 X2330 Black', 444, Decimal('550'))
('WWI Desktop PC1.80 E1800 White', 458, Decimal('538'))
('Adventure Works Desktop PC1.60 ED160 Black', 424, Decimal('521'))
('Adventure Works Desktop PC2.30 MD230 White', 434, Decimal('521'))
('Adventure Works Desktop PC1.80 ED180 Black', 425, Decimal('520'))


In [26]:
# 10. Product Popularity: Identify the most and least popular products based on sales data.

# identifying least popular product based on total quantity sold

query = """SELECT 
    p.Product_Name,
    p.ProductKey,
    SUM(s.Quantity) AS TotalQuantitySold
FROM 
    products p
JOIN 
    sales s ON p.ProductKey = s.ProductKey
GROUP BY 
    p.ProductKey, p.Product_Name
ORDER BY 
    TotalQuantitySold ASC
LIMIT 5;"""

cursor.execute(query)

results = cursor.fetchall()

# Fetching the column names
column_names = [i[0] for i in cursor.description]

# Printing the column names
print(column_names)
for row in results:
    print(row)

['Product_Name', 'ProductKey', 'TotalQuantitySold']
('WWI Wall Lamp E315 White', 2243, Decimal('1'))
('Adventure Works Desk Lamp E1300 Black', 2196, Decimal('1'))
('Proseware Wall Lamp E0215 Blue', 2306, Decimal('1'))
('WWI Floor Lamp X115 Blue', 2262, Decimal('1'))
('WWI Floor Lamp M215 Grey', 2255, Decimal('1'))


In [18]:
# 11. Profitability Analysis: Calculate profit margins for products by comparing unit cost and unit price.
# sql query to display productkey, product name, total profit, total revenue and profit margins
# of products having highest profit margins

query = """WITH product_profits AS (
    SELECT 
        p.ProductKey,
        p.Product_Name,
        SUM(s.Quantity * (p.Unit_Price_USD - p.Unit_Cost_USD)) AS TotalProfit,
        SUM(s.Quantity * p.Unit_Price_USD) AS TotalRevenue,
        (SUM(s.Quantity * (p.Unit_Price_USD - p.Unit_Cost_USD)) / (SUM(s.Quantity * p.Unit_Price_USD))) * 100 AS ProfitMargin
    FROM 
        products p
    JOIN 
        sales s ON p.ProductKey = s.ProductKey
    GROUP BY 
        p.ProductKey, p.Product_Name
),
ranked_products AS (
    SELECT 
        *,
        RANK() OVER (ORDER BY ProfitMargin DESC) AS ProfitMarginRank
    FROM 
        product_profits
)
SELECT 
    ProductKey,
    Product_Name,
    TotalProfit,
    TotalRevenue,
    ProfitMargin
FROM 
    ranked_products
WHERE 
    ProfitMarginRank = 1
ORDER BY 
    TotalProfit DESC;
"""
cursor.execute(query)

results = cursor.fetchall()

# Fetching the column names
column_names = [i[0] for i in cursor.description]

# Printing the column names
print(column_names)
for row in results:
    print(row)

['ProductKey', 'Product_Name', 'TotalProfit', 'TotalRevenue', 'ProfitMargin']
(1635, 'Contoso DVD 60 DVD Storage Binder L20 Silver', Decimal('4761.41'), Decimal('7118.79'), Decimal('66.885103'))
(1590, 'SV DVD 60 DVD Storage Binder L20 Silver', Decimal('4746.10'), Decimal('7095.90'), Decimal('66.885103'))
(1585, 'SV DVD 60 DVD Storage Binder L20 Black', Decimal('4669.55'), Decimal('6981.45'), Decimal('66.885103'))
(1595, 'SV DVD 60 DVD Storage Binder L20 Red', Decimal('4240.87'), Decimal('6340.53'), Decimal('66.885103'))
(1640, 'Contoso DVD 60 DVD Storage Binder L20 Red', Decimal('4072.46'), Decimal('6088.74'), Decimal('66.885103'))
(1630, 'Contoso DVD 60 DVD Storage Binder L20 Black', Decimal('3888.74'), Decimal('5814.06'), Decimal('66.885103'))


In [20]:
# 12. Category Analysis: Analyze sales performance across different product categories and subcategories.

# sql query to get total revenue across different product categories and subcategories

query = """SELECT 
    p.Category,
    p.Subcategory,
    SUM(s.Quantity * p.Unit_Price_USD) AS TotalRevenue
FROM 
    products p
LEFT JOIN 
    sales s ON p.ProductKey = s.ProductKey
GROUP BY 
    p.Category,
    p.Subcategory
ORDER BY 
    p.Category,
    p.Subcategory,
    TotalRevenue DESC;"""

cursor.execute(query)

results = cursor.fetchall()

# Fetching the column names
column_names = [i[0] for i in cursor.description]

# Printing the column names
print(column_names)
for row in results:
    print(row)

['Category', 'Subcategory', 'TotalRevenue']
('Audio', 'Bluetooth Headphones', Decimal('1380400.74'))
('Audio', 'MP4&MP3', Decimal('478802.45'))
('Audio', 'Recording Pen', Decimal('1310424.55'))
('Cameras and camcorders', 'Camcorders', Decimal('3357990.00'))
('Cameras and camcorders', 'Cameras & Camcorders Accessories', Decimal('381856.82'))
('Cameras and camcorders', 'Digital Cameras', Decimal('885679.80'))
('Cameras and camcorders', 'Digital SLR Cameras', Decimal('1894641.40'))
('Cell phones', 'Cell phones Accessories', Decimal('128948.34'))
('Cell phones', 'Home & Office Phones', Decimal('165723.88'))
('Cell phones', 'Smart phones & PDAs', Decimal('2805657.00'))
('Cell phones', 'Touch Screen Phones', Decimal('3083462.00'))
('Computers', 'Computers Accessories', Decimal('211345.76'))
('Computers', 'Desktops', Decimal('9906356.50'))
('Computers', 'Laptops', Decimal('3164777.20'))
('Computers', 'Monitors', Decimal('1518116.00'))
('Computers', 'Printers, Scanners & Fax', Decimal('733478.

In [21]:
# 13. Geographical Analysis: Analyze sales by store location to identify high-performing regions.

query = """SELECT 
    st.Country_Store,
    SUM(s.Quantity * p.Unit_Price_USD) AS TotalRevenue
FROM 
    sales s
JOIN 
    products p ON s.ProductKey = p.ProductKey
JOIN 
    stores st ON s.StoreKey = st.StoreKey
GROUP BY 
    st.Country_Store
ORDER BY 
    TotalRevenue DESC;"""

cursor.execute(query)

results = cursor.fetchall()

# Fetching the column names
column_names = [i[0] for i in cursor.description]

# Printing the column names
print(column_names)
for row in results:
    print(row)

['Country_Store', 'TotalRevenue']
('United States', Decimal('47528851.72'))
('United Kingdom', Decimal('11499539.56'))
('Online', Decimal('11404324.63'))
('Germany', Decimal('8492558.44'))
('Canada', Decimal('7223123.58'))
('Australia', Decimal('4198282.14'))
('Italy', Decimal('4118173.62'))
('Netherlands', Decimal('3182688.96'))
('France', Decimal('2459091.90'))


In [27]:
cursor.close()
con.close()